In [1]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/rocket/anaconda2/h2o_jar/h2o.jar
  Ice root: /tmp/tmpqwcQYq
  JVM stdout: /tmp/tmpqwcQYq/h2o_rocket_started_from_python.out
  JVM stderr: /tmp/tmpqwcQYq/h2o_rocket_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,"1 year, 2 months and 28 days !!!"
H2O cluster name:,H2O_from_python_rocket_dw9mc9
H2O cluster total nodes:,1
H2O cluster free memory:,3.464 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


# Data

In [2]:
df = h2o.import_file(path="../datasets/wine.csv")
df.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:1599
Cols:12




,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
type,real,real,real,real,real,real,real,real,real,real,real,int
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0
mean,8.3196372733,0.527820512821,0.270975609756,2.53880550344,0.0874665415885,15.8749218261,46.4677923702,0.996746679174,3.31111319575,0.658148843027,10.4229831144,5.63602251407
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9,8.0
sigma,1.74109631813,0.179059704154,0.194801137405,1.40992805951,0.0470653020101,10.4601569698,32.8953244783,0.00188733395384,0.154386464904,0.16950697959,1.06566758185,0.807569439735
zeros,0,0,132,0,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5.0


In [3]:
# Define predictors
y = df['quality']
X = df
X = X.drop(['quality'])
X.describe()
y.describe()

Rows:1599
Cols:11




,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
type,real,real,real,real,real,real,real,real,real,real,real
mins,4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4
mean,8.3196372733,0.527820512821,0.270975609756,2.53880550344,0.0874665415885,15.8749218261,46.4677923702,0.996746679174,3.31111319575,0.658148843027,10.4229831144
maxs,15.9,1.58,1.0,15.5,0.611,72.0,289.0,1.00369,4.01,2.0,14.9
sigma,1.74109631813,0.179059704154,0.194801137405,1.40992805951,0.0470653020101,10.4601569698,32.8953244783,0.00188733395384,0.154386464904,0.16950697959,1.06566758185
zeros,0,0,132,0,0,0,0,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8


Rows:1599
Cols:1




,quality
type,int
mins,3.0
mean,5.63602251407
maxs,8.0
sigma,0.807569439735
zeros,0
missing,0
0,5.0
1,5.0
2,5.0


In [4]:
# Split for training
train, valid, test = df.split_frame(
    ratios = [0.6, 0.2],
    seed=4567,
    destination_frames=['train.hex','valid.hex','test.hex']
)

# Experiments

## Single Decision Tree (No Boost)

In [5]:
# Single decision tree, no boosting single tree
rfe = H2ORandomForestEstimator(ntrees=1)
rfe.train(x=X.columns, y=y.columns[0], training_frame=train)

# Show model summary
print rfe

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1517511809885_1
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,1.0,1.0,2986.0,16.0,16.0,16.0,232.0,232.0,232.0




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.924855491329
RMSE: 0.961694073669
MAE: 0.624277456647
RMSLE: 0.150768517168
Mean Residual Deviance: 0.924855491329
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-02-01 14:03:46,0.117 sec,0.0,nan,nan,nan
,2018-02-01 14:03:47,0.643 sec,1.0,0.9616941,0.6242775,0.9248555


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
total sulfur dioxide,56.1954079,1.0,0.1382328
sulphates,53.1694489,0.9461529,0.1307893
density,52.6726265,0.9373119,0.1295672
alcohol,48.3738594,0.8608152,0.1189929
free sulfur dioxide,42.3137207,0.7529747,0.1040858
volatile acidity,33.3538322,0.5935331,0.0820457
fixed acidity,31.3587208,0.5580300,0.0771380
citric acid,30.6309738,0.5450797,0.0753479
residual sugar,23.8561211,0.4245208,0.0586827
chlorides,21.2128334,0.3774834,0.0521806


In [6]:
perf = rfe.model_performance(valid)
perf


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.68253968254
RMSE: 0.826159598709
MAE: 0.536507936508
RMSLE: 0.126906538368
Mean Residual Deviance: 0.68253968254


## Cross-Validation Tuning

In [7]:
# Lets see if we can do better on one tree
hyper_parameters = {'ntrees':[1], 
                    'max_depth':[20, 5], 
                    'balance_classes':[True, False], 
                    'distribution':['bernoulli', 'multinominal', 'gaussian', 'poisson', 'gamma', 'tweedie', 'laplace', 'huber'],
                    'histogram_type':['uniform_adaptive', 'random', 'quantiles_global', 'round_robin']}
grid_search = H2OGridSearch(H2ORandomForestEstimator, hyper_params=hyper_parameters)
grid_search.train(x=X.columns, y=y.columns[0], training_frame=train)
grid_search.show()

drf Grid Build progress: |████████████████████████████████████████████████| 100%
      balance_classes distribution   histogram_type max_depth ntrees  \
0               false      poisson  UniformAdaptive        20      1   
1               false      poisson       RoundRobin        20      1   
2               false      poisson  UniformAdaptive         5      1   
3               false      poisson  QuantilesGlobal         5      1   
4                true      poisson           Random         5      1   
5               false      poisson           Random         5      1   
6                true      poisson       RoundRobin         5      1   
7                true      poisson  UniformAdaptive         5      1   
8                true      poisson  QuantilesGlobal         5      1   
9                true      poisson       RoundRobin        20      1   
10              false      poisson           Random        20      1   
11               true      poisson           Random    

[80 rows x 8 columns]


In [9]:
# Now select the configuration with the lowest logloss
opt_tree = grid_search.get_grid(sort_by='rmse', decreasing=True)[-1]

In [10]:
opt_tree.model_performance(valid)


ModelMetricsRegression: drf
** Reported on test data. **

MSE: 0.551415568347
RMSE: 0.7425736114
MAE: 0.563984332009
RMSLE: 0.114646877236
Mean Residual Deviance: 0.0424166403545


In [11]:
h2o.shutdown()

H2O session _sid_b64b closed.
